In [1]:
#imports
import lxml.etree as ET
import os

In [2]:
# Applies XSLT transformations to extract the necessary information from the Aletheia PAGE XML
# The information is then processed to obtain a Tesseract box file

def xml_to_box(inputFile, outputFile):
    with open(inputFile, encoding="utf8") as f:
        dom = ET.parse(f)

    with open("get_symbol.xsl", encoding="utf8") as f:
        xslt = ET.parse(f)
    transform = ET.XSLT(xslt)
    newdom = transform(dom)
    sym = str(newdom, 'utf8')

    with open("get_coordinates.xsl", encoding="utf8") as f:
        xslt = ET.parse(f)
    transform = ET.XSLT(xslt)
    newdom = transform(dom)
    coord = str(newdom, 'utf8')

    with open("get_height.xsl", encoding="utf8") as f:
        xslt = ET.parse(f)
    transform = ET.XSLT(xslt)
    newdom = transform(dom)
    imageHeight = int(str(newdom, 'utf8'))

    # Create list of coordinates for each symbol
    a = coord.split('\n')
    # strip last line if empty
    if a[-1] == '':
        del a[-1]

    # Create list of symbols
    b = sym.split()

    # initialise box data coordinate list
    box = []

    for i in range(len(a)):
        # create list of coordinate pairs
        s = a[i].split(' ')

        # save coordinate pairs needed
        c1 = s[0].split(',')
        c2 = s[1].split(',')
        c4 = s[3].split(',')

        # get box coordinates
        x1 = int(c1[0]) # X
        y1 = int(c1[1]) # Y
        height = int(c2[1])-y1
        width = int(c4[0])-x1

        #print('x1 = {x1}; y1={y1}; width = {w}; height = {h}'.format(x1=x1,y1=y1, h=height, w= width))

        # create box data (coordinate section)
        bottom = (imageHeight-y1)-height
        right = x1 + width
        top = imageHeight - y1

        # add to list
        # Since we will be processing a single page, '0' is added to the end of each line
        box.append(b[i] + ' ' + str(x1) + ' ' + str(bottom) + ' ' + str(right) + ' ' + str(top) + ' 0' )

    # create output text file

    with open(outputFile, 'w', encoding="utf8") as filehandle:
        for i in box:
            filehandle.write('%s\n' % i)


In [5]:
# Change Directory if needed
# os.chdir(<path>)